In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import pytz
from IPython.display import VimeoVideo
from pymongo import MongoClient
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [ ]:
host = "192.104.104.2"

In [ ]:
client = MongoClient(host=host, port=27017)
db = client["air-quality"]
nairobi = db["nairobi"]

In [ ]:
def wrangle(collection):
    results = collection.find(
        {"metadata.site": 29, "metadata.measurement": "P2"},
        projection={"P2": 1, "timestamp": 1, "_id": 0},
    )
    df = pd.DataFrame(results).set_index("timestamp")
    #localize timezone
    df.index= df.index.tz_localize("UTC").tz_convert("Africa/Nairobi")
    #Remove outlier
    df=df[df["P2"] < 500]
    # resample with msissing data and null values
    df= df["P2"].resample("1H").mean().fillna(method="ffill").to_frame()

    # add lag
    df["P2.L1"]=df["P2"].shift(1)

    # drop na
    df.dropna(inplace=True)
    return df

In [ ]:
df = wrangle(nairobi)
df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

df["P2"].plot(kind="box", vert=False, title="Distribution of P2.5 Readings", ax=ax);

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

df["P2"].plot(xlabel="Time" , ylabel="PM2.5", title="PM2.5 time serise", ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

df["P2"].rolling(168).mean().plot(xlabel="Time" , ylabel="PM2.5", title="Weekly rolling avg", ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

ax.scatter(x=df["P2.L1"], y= df["P2"])
ax.plot([0,120],[0,120],linestyle="--",color="orange")
plt.xlabel("P2.L1")
plt.ylabel("P2")
plt.title("PM2.5 Autocorrelation")

In [ ]:
target = "P2"
y = df[target]
X = df.drop(columns=target)
X.head()

In [ ]:
cutoff = int(len(X)*0.8)

X_train, y_train = X.iloc[:cutoff] , y.iloc[:cutoff]
X_test, y_test = X.iloc[cutoff:] , y.iloc[cutoff:]

In [ ]:
y_pred_baseline = [y_train.mean()] * len(y_train)
mae_baseline = mean_absolute_error(y_train,y_pred_baseline)

print("Mean P2 Reading:", round(y_train.mean(), 2))
print("Baseline MAE:", round(mae_baseline, 2))

In [ ]:
model= LinearRegression()
model.fit(X_train,y_train)

In [ ]:
training_mae = mean_absolute_error(y_train,model.predict(X_train))
test_mae = mean_absolute_error(y_test,model.predict(X_test))
print("Training MAE:", round(training_mae, 2))
print("Test MAE:", round(test_mae, 2))

In [ ]:
intercept = model.intercept_.round(2)
coefficient = model.coef_.round(2)[0]

print(f"P2 = {intercept} + ({coefficient} * P2.L1)")

In [ ]:
df_pred_test = pd.DataFrame(
    {
        "y_test" : y_test,
        "y_pred" : model.predict(X_test)
    }
)
df_pred_test.head()

In [ ]:
fig = px.line(df_pred_test, labels={"value":"P2"})
fig.show()